In [1]:
from matplotlib import pyplot as plt
from nltk.tokenize import sent_tokenize
import pandas as pd
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')
import numpy as np
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df=pd.read_csv('sentences.csv')
summ=df.head(100)
summ.head()


,task,paper_id,doi,title,summary,score,sentences
0,what is the immune system response to 2019-ncov ?,40dabecefc2f0a2b750e000d1c15cca57a3f7dd4,10.1016/0163-7258(85)90033-6,ANIMAL VIRUSES OF ECONOMIC IMPORTANCE: GENETIC...,in broad terms the immune system among vertebr...,0.927845,[('in broad terms the immune system among vert...
1,what is the immune system response to 2019-ncov ?,2008f024274c53b0de1276f4047a6a641813fd98,10.1016/S1081-1206(10)61294-X,ABSTRACTS: CONCURRENT SESSIONS,the infection has also occurred in newborns wi...,0.814046,[('the infection has also occurred in newborns...
2,what is the immune system response to 2019-ncov ?,fea50730d7cdcd7c728f98c31fe20aeb8af5e350,10.1016/j.compbiomed.2020.103670,Computers and viral diseases. Preliminary bioi...,is 2019-ncov. cysteine c should be avoided per...,0.779853,"[('is 2019-ncov.', 1.0743433338565338), ('cyst..."
3,what is the immune system response to 2019-ncov ?,6aadca94314fe7e81e278fbbef178f2a5bf4f538,10.1186/cc13531,Minimising prescribing errors in the ICU DJ Me...,also in these patients cortisol correlates to ...,0.763274,[('also in these patients cortisol correlates ...
4,what is the immune system response to 2019-ncov ?,68ac63121bded12e1db3178a0c9b050154f81ab8,10.1016/j.mehy.2015.11.005,Seasonality and selective trends in viral acut...,although heat stress may boost the immune syst...,0.724360,[('although heat stress may boost the immune s...


In [3]:
summ.shape

(100, 7)

In [4]:
def remove_stopwords(sen):     
    sen_new = " ".join([i for i in sen if i not in stop_words])          
    return sen_new

In [5]:
i=1
sentences = [] 
for s in summ['summary']: 
                   
        for x in(sent_tokenize(s)) :
            sentences.append(sent_tokenize(x))
#print(sentences)    
corpus = [y for x in sentences for y in x ]
#print(corpus)


clean_sentences = pd.Series(corpus).str.replace("[^a-zA-Z]", " ") 
        # make alphabets lowercase 

clean_sentences = [s.lower() for s in clean_sentences]
clean_sentences = [remove_stopwords(r.split()) for r in corpus]


In [6]:

# Extract word vectors 
word_embeddings = {} 
f = open(r"C:\meriem\Mémoire\glove6b\glove.6B.100d.txt", encoding='utf-8') 
for line in f: 
    values = line.split() 
    word = values[0] 
    coefs = np.asarray(values[1:], dtype='float32')    
    word_embeddings[word] = coefs 
f.close()

In [7]:
sentence_vectors = [] 
for i in clean_sentences: 
    if len(i) != 0: 
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001) 
    else: 
        v = np.zeros((100,)) 
    sentence_vectors.append(v)

In [8]:
sim_mat = np.zeros([len(corpus), len(corpus)])
for i in range(len(corpus)): 
      for j in range(len(corpus)): 
        if i != j: 
              sim_mat[i][j] = cosine_similarity (sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [9]:
import networkx as nx 
nx_graph = nx.from_numpy_array(sim_mat) 
scores = nx.pagerank(nx_graph)

In [11]:

ranked_sentences = sorted(((scores[i],s) for i,s in 
                           enumerate(corpus)), reverse=True)
# Extract top 10 sentences as the summary 
for i in range(20): 
      print(ranked_sentences[i][1])

the adaptive immune response to rv infection has been well characterized and studies have demonstrated that cd8+ t cells play a critical role in the timely clearance of a primary infection while b cells are crucial for resistance to re-infection .
in terms of the human immune system the innate immune system response plays an important role in controlling the replication and infection of coronavirus and interferon is expected to enhance the immune response 11 .
in the present work we have tried to contribute to the information on the immune mechanisms occurring after pedv infection by making a ®rst approach to the characterisation of the antibody response generated by the virus.
however for several reasons many virus vector systems are still limited in their ability to induce a broad and long-lasting antiviral immune response capable to prevent hiv infection and/or to reduce viral load.
the study shows that the functions of proteinases in the virus entry into the cell as well as in host